In [8]:
import os
import numpy as np
import librosa
from hmmlearn import hmm
from sklearn.model_selection import KFold

In [9]:
# Function to extract MFCC features from an audio file
def extract_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=None)  # Load the audio file
    mfcc_features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # Extract MFCC features
    return mfcc_features.T  # Transpose for HMM compatibility

In [10]:
# Function to train an HMM for a given feature set
def train_hmm(features_list):
    lengths = [len(f) for f in features_list]  # Sequence lengths
    X = np.vstack(features_list)  # Combine all feature sequences
    model = hmm.GaussianHMM(n_components=4, covariance_type="full", n_iter=100)  # Define HMM
    model.fit(X, lengths)  # Train HMM
    return model

In [11]:
# Function to predict class of test features
def predict_hmm(models, test_features):
    scores = {cls: model.score(test_features) for cls, model in models.items()}  # Compute log-likelihood
    return max(scores, key=scores.get)  # Return class with highest likelihood

In [12]:
#Function to predict class of test features
def predict_hmm(models, test_features):
    scores = {cls: model.score(test_features) for cls, model in models.items()}  # Compute log-likelihood
    return max(scores, key=scores.get)  # Return class with highest likelihood

In [14]:
# Step 1: Load Data from Train and Test Directories
# Step 1: Data Preparation
data_dir = "E://sastra//MiniProject//b14//review2//mydigits"  # Replace with your dataset directory
classes = ["D0", "D1", "D2","D3","D4","D5","D6","D7","D8","D9"]  # Replace with your class labels
training_data = {cls: [] for cls in classes}

for cls in classes:
        class_dir = os.path.join(data_dir, cls)
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            features = extract_features(file_path)
            training_data[cls].append(features)

all_data = []  # Store (features, label) tuples
all_labels = []  # Store labels separately

for cls in classes:
    class_dir = os.path.join(data_dir, cls)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        features = extract_features(file_path)
        all_data.append(features)
        all_labels.append(cls)

# Convert lists to numpy arrays
all_data = np.array(all_data, dtype=object)
all_labels = np.array(all_labels)

# Step 2: K-Fold Cross-Validation
k_folds = 5  # Define number of folds
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)  # Shuffle for randomness

accuracies = []

for fold, (train_idx, test_idx) in enumerate(kf.split(all_data), 1):
    print(f"\n=== Fold {fold}/{k_folds} ===")
    
    # Split data into training and testing
    train_data, test_data = all_data[train_idx], all_data[test_idx]
    train_labels, test_labels = all_labels[train_idx], all_labels[test_idx]

    # Train HMM models for each class
    models = {}
    for cls in np.unique(train_labels):
        cls_features = [train_data[i] for i in range(len(train_data)) if train_labels[i] == cls]
        print(f"Training HMM for class: {cls}")
        models[cls] = train_hmm(cls_features)

    # Evaluate on test data
    correct = 0
    total = 0

    for i, test_features in enumerate(test_data):
        true_label = test_labels[i]
        predicted_label = predict_hmm(models, test_features)

        if predicted_label == true_label:
            correct += 1
        total += 1

        print(f"Test Sample {i+1}: True Label = {true_label}, Predicted = {predicted_label}")

    # Compute fold accuracy
    fold_accuracy = (correct / total) * 100
    accuracies.append(fold_accuracy)
    print(f"Fold {fold} Accuracy: {fold_accuracy:.2f}%")


C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,



=== Fold 1/5 ===
Training HMM for class: D0
Training HMM for class: D1
Training HMM for class: D2
Training HMM for class: D3
Training HMM for class: D4
Training HMM for class: D5
Training HMM for class: D6
Training HMM for class: D7
Training HMM for class: D8
Training HMM for class: D9
Test Sample 1: True Label = D0, Predicted = D0
Test Sample 2: True Label = D0, Predicted = D0
Test Sample 3: True Label = D0, Predicted = D0
Test Sample 4: True Label = D0, Predicted = D0
Test Sample 5: True Label = D0, Predicted = D3
Test Sample 6: True Label = D0, Predicted = D9
Test Sample 7: True Label = D0, Predicted = D0
Test Sample 8: True Label = D0, Predicted = D0
Test Sample 9: True Label = D0, Predicted = D0
Test Sample 10: True Label = D0, Predicted = D0
Test Sample 11: True Label = D0, Predicted = D0
Test Sample 12: True Label = D0, Predicted = D0
Test Sample 13: True Label = D0, Predicted = D1
Test Sample 14: True Label = D0, Predicted = D1
Test Sample 15: True Label = D0, Predicted = D0
T

In [ ]:
# Compute and print overall accuracy
final_accuracy = np.mean(accuracies)
print(f"\n=== Final K-Fold Cross-Validation Accuracy: {final_accuracy:.2f}% ===")